# How to revoke tokens from Vault

In this tutorial, we cover how to revoke a lot of tokens at once.

Here are the high level steps:

1. Get a list of accessors on a namespace.
    1. I set an arbitrary limit of `100` due to concerns of load.
    1. If you have millions of accessors, then you might want to increase this.
1. Create a filtered list for revoking.
    1. For each accessor, get relevant information such as policies, ttl, accessor, etc
    1. Filter out unwanted items. Our example filters out:
        - display name is `root` - don't want to delete root token
        - ttl is less than an hour - let these age out on their own
        - has the words `admin` or `root` - replace these with your own
1. Revoke tokens based on list

You might need to do this if you:
- accidentally created a lot of tokens via automation
- have long-lived tokens
- reaching lease limits
- impacting system resources

## Prerequisites

### vault server - optional

Spin up a dev Vault server.

Specify vault enterprise version and license.
- If you don't have a license, then don't run the following commands.
- It will default to Vault version 1.7 which includes a 6 hour eval license.

In [ ]:
VAULT_VERSION=1.10-ent
export VAULT_LICENSE=$(cat ./vault.hclic)

Spin up Vault server container running in dev mode.

In [7]:
mkdir -p /tmp/vault
cat > /tmp/vault/config.hcl <<EOF
# storage "raft" {
#   path    = "./vault/data"
#   node_id = "node1"
# }
storage "file" {
  path = "/vault/file"
}

listener "tcp" {
  address     = "0.0.0.0:8200"
  tls_disable = "true"
}

disable_mlock = true

api_addr = "http://127.0.0.1:8200"
cluster_addr = "https://127.0.0.1:8201"
ui = true
EOF

In [9]:
docker stop vault && docker rm vault
# docker run --rm -d \
docker run -d \
    --name vault \
    -p 8200:8200 \
    -e "VAULT_LICENSE=$VAULT_LICENSE" \
    -e 'VAULT_DEV_ROOT_TOKEN_ID=root' \
    -e "VAULT_ADDR=http://127.0.0.1:8200" \
    --cap-add=IPC_LOCK \
    -v /tmp/vault/config.hcl:/vault/config/config.hcl \
    hashicorp/vault-enterprise:${VAULT_VERSION:-1.7-ent} \
    server
docker ps | grep vault

Error response from daemon: No such container: vault
ff4c7b6db1fee1e73a488f7c09c99e8b721dd040dbee8fab37d0a7eba2ba1555
ff4c7b6db1fe   hashicorp/vault-enterprise:1.7-ent   "docker-entrypoint.s…"   1 second ago   Up Less than a second   0.0.0.0:8200->8200/tcp, :::8200->8200/tcp                                                        vault


In [40]:
#==> Prepare environment variables:
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_TOKEN=root
unset VAULT_NAMESPACE

Save the unseal key(s) and root token.

In [14]:
vault operator init -format=json \
  -key-shares=1 -key-threshold=1 | tee /tmp/vault/vault.init

In [35]:
export VAULT_TOKEN=$(jq -r .root_token /tmp/vault/vault.init)
echo $VAULT_TOKEN

s.pwjXATOMWpo9KM56mLMXamf3


In [29]:
VAULT_UNSEAL_KEY1=$(jq -r .unseal_keys_b64[0] /tmp/vault/vault.init) && echo $VAULT_UNSEAL_KEY1

s.pwjXATOMWpo9KM56mLMXamf3
/Xm2ZOX0UxaWdQZENll/kYrdPeMM9Gp1HPK+UKqY1vc=


In [23]:
vault operator unseal $VAULT_UNSEAL_KEY1

Key             Value
---             -----
Seal Type       shamir
Initialized     true
Sealed          false
Total Shares    1
Threshold       1
Version         1.7.10+ent
Storage Type    file
Cluster Name    vault-cluster-269b24c8
Cluster ID      e0860d4c-2db7-e75b-8559-9f3ac839b0e6
HA Enabled      false


In [22]:
cat /tmp/vault/vault.init

{
  "unseal_keys_b64": [
    "/Xm2ZOX0UxaWdQZENll/kYrdPeMM9Gp1HPK+UKqY1vc="
  ],
  "unseal_keys_hex": [
    "fd79b664e5f453169675064436597f918add3de30cf46a751cf2be50aa98d6f7"
  ],
  "unseal_shares": 1,
  "unseal_threshold": 1,
  "recovery_keys_b64": [],
  "recovery_keys_hex": [],
  "recovery_keys_shares": 5,
  "recovery_keys_threshold": 3,
  "root_token": "s.pwjXATOMWpo9KM56mLMXamf3"
}


Verify

In [42]:
vault status; echo; vault secrets list

Key             Value
---             -----
Seal Type       shamir
Initialized     true
Sealed          false
Total Shares    1
Threshold       1
Version         1.7.10+ent
Storage Type    file
Cluster Name    vault-cluster-269b24c8
Cluster ID      e0860d4c-2db7-e75b-8559-9f3ac839b0e6
HA Enabled      false

Path          Type         Accessor              Description
----          ----         --------              -----------
cubbyhole/    cubbyhole    cubbyhole_d2410794    per-token private secret storage
identity/     identity     identity_32981e20     identity store
sys/          system       system_ac78f3d1       system endpoints used for control, policy and debugging


### namespace - optional

In [47]:
VAULT_NAMESPACE_NS1=coke

In [49]:
vault namespace create $VAULT_NAMESPACE_NS1
export VAULT_NAMESPACE=$VAULT_NAMESPACE_NS1

Key     Value
---     -----
id      Pubjt
path    coke/


In [ ]:
vault secrets list

## Steps

### create vault tokens<a name=create_vault_tokens></a>

- 2000 tokens takes 34 seconds

API option

In [154]:
printf "#==> Create test tokens. ttl: 2h. policy: dev ops.\n"

time for x in {1..2000}; do
  # vault token create -ttl=2h -policy=dev > /dev/null 2>&1
# vault token create -ttl=90m -policy=ops > /dev/null 2>&1
curl -s \
  --header "X-Vault-Token: $VAULT_TOKEN" \
  --header "X-Vault-Namespace: $VAULT_NAMESPACE" \
  --request POST \
  --data '{"policies": ["dev"],"ttl":"2h"}' \
  ${VAULT_ADDR}/v1/auth/token/create > /dev/null
curl -s \
  --header "X-Vault-Token: $VAULT_TOKEN" \
  --header "X-Vault-Namespace: $VAULT_NAMESPACE" \
  --request POST \
  --data '{"policies": ["ops"],"ttl":"2h"}' \
  ${VAULT_ADDR}/v1/auth/token/create > /dev/null
done

printf "#==> Create test tokens. ttl: 90m. policy: admin ops.\n"
time for x in {1..5}; do
  # vault token create -ttl=90m -policy=ops -policy=admin > /dev/null 2>&1
  # vault token create -ttl=90m -policy=admin > /dev/null 2>&1
curl -s \
  --header "X-Vault-Token: $VAULT_TOKEN" \
  --header "X-Vault-Namespace: $VAULT_NAMESPACE" \
  --request POST \
  --data '{"policies": ["ops","admin"],"ttl":"90m"}' \
  ${VAULT_ADDR}/v1/auth/token/create > /dev/null
curl -s \
  --header "X-Vault-Token: $VAULT_TOKEN" \
  --header "X-Vault-Namespace: $VAULT_NAMESPACE" \
  --request POST \
  --data '{"policies": ["admin"],"ttl":"90m"}' \
  ${VAULT_ADDR}/v1/auth/token/create > /dev/null

done

#==> Create test tokens. ttl: 2h. policy: dev ops.

real	0m34.129s
user	0m13.431s
sys	0m15.283s
#==> Create test tokens. ttl: 90m. policy: admin ops.

real	0m0.054s
user	0m0.033s
sys	0m0.013s


CLI options

```shell
printf "#==> Create test tokens. ttl: 2h. policy: dev ops.\n"

time for x in {1..20}; do
  vault token create -ttl=2s -policy=dev > /dev/null 2>&1
  vault token create -ttl=2s -policy=ops > /dev/null 2>&1
done

printf "#==> Create test tokens. ttl: 90m. policy: admin ops.\n"
time for x in {1..5}; do
  vault token create -ttl=2s -policy=ops -policy=admin > /dev/null 2>&1
  vault token create -ttl=2s -policy=admin > /dev/null 2>&1
done
```

### Get initial list of accessors

Send specified number (`1000`) of accessors to a list.

API option

In [195]:
printf "\n#==> Get list of token accessors and place into variable\n"
time curl -s \
  --header "X-Vault-Token: $VAULT_TOKEN" \
  --header "X-Vault-Namespace: $VAULT_NAMESPACE" \
  --request LIST \
  ${VAULT_ADDR}/v1/auth/token/accessors \
  | jq -r .data.keys[] \
  > /tmp/vault_accessors.out


head -n 1000 /tmp/vault_accessors.out > /tmp/vault_accessors_short.out


printf "\n#==> Quick check of list\n"
printf "Lines in list: $(cat /tmp/vault_accessors_short.out | wc -l)\n"
head /tmp/vault_accessors_short.out


#==> Get list of token accessors and place into variable

real	0m0.037s
user	0m0.026s
sys	0m0.000s

#==> Quick check of list
Lines in list: 1000
HFm4s0bhJPP6a1hI99nUCJtq.Pubjt
8LeXP5XKdfZ9aaJjkPXBh6Av.Pubjt
LHGuRMJieaxBMwtH93KL8GBq.Pubjt
27EIBiwivH30W14LVpYMxbya.Pubjt
FV44XiYgcERyYLFWnA7rNzdY.Pubjt
zZzI5ldkXptUVN0UoXgz9NJ7.Pubjt
dYsoNQUZO9kxmG4ySTlfNZTP.Pubjt
F979MwQBYxVmUHhU2BocgKXx.Pubjt
56TIiaEnF7woIRba8wZotOmb.Pubjt
Z4xk9t8rlWfOkYvNe7ocKUnh.Pubjt


CLI Option

```shell
printf "\n#==> Get list of token accessors and place into variable\n"
time vault list auth/token/accessors | head -n 102 | tail -n +3 \
  > /tmp/vault_accessors.out

printf "\n#==> Quick check of list\n"
printf "Lines in list: $(cat /tmp/vault_accessors.out | wc -l)\n"
head /tmp/vault_accessors.out
```

### Create a filtered list

Create a file that contains json of accessors that don't have the `admin` or `root` policy.
- You can replace that with other policies that you don't want to include.
- In our example, only tokens that have `dev` but not `admin` or `root` will be included.
- show display_name, creation_time, policies, accessor, and ttl (converted to hours).

API option

In [196]:
printf "\n#==> Lookup accessors in variable and save data to file.\n"
printf "#==> Exclude tokens with root policy.\n\n"
time while read accessor; do
curl -s \
  --header "X-Vault-Token: $VAULT_TOKEN" \
  --header "X-Vault-Namespace: $VAULT_NAMESPACE" \
  --request POST \
  --data "{\"accessor\": \"$accessor\"}" \
  ${VAULT_ADDR}/v1/auth/token/lookup-accessor \
  | jq -c .data \
  | grep -Ev "admin|root" \
  | jq -c 'select((.display_name == "root"|not) and .ttl > 60 ) ' \
  | jq -c '{Display_Name: .display_name, 
  Creation_Time:(.creation_time | strflocaltime("%Y-%m-%dT%H:%M")),
  Policies: .policies, Accessor: .accessor, ttl: (.ttl/60 | round ) }'
done < /tmp/vault_accessors_short.out | tee /tmp/vault_accessors_delete.out > /dev/null

printf "\n#==> Accessors in list: $(cat /tmp/vault_accessors_delete.out | wc -l)\n"
printf "\n#==> Quick check of list\n"
head /tmp/vault_accessors_delete.out


#==> Lookup accessors in variable and save data to file.
#==> Exclude tokens with root policy.


real	0m40.246s
user	1m9.420s
sys	0m13.659s

#==> Accessors in list: 984

#==> Quick check of list
{"Display_Name":"token","Creation_Time":"2022-07-25T16:52","Policies":["default","dev"],"Accessor":"FV44XiYgcERyYLFWnA7rNzdY.Pubjt","ttl":61}
{"Display_Name":"token","Creation_Time":"2022-07-25T16:52","Policies":["default","dev"],"Accessor":"zZzI5ldkXptUVN0UoXgz9NJ7.Pubjt","ttl":61}
{"Display_Name":"token","Creation_Time":"2022-07-25T16:51","Policies":["default","ops"],"Accessor":"dYsoNQUZO9kxmG4ySTlfNZTP.Pubjt","ttl":60}
{"Display_Name":"token","Creation_Time":"2022-07-25T16:51","Policies":["default","dev"],"Accessor":"F979MwQBYxVmUHhU2BocgKXx.Pubjt","ttl":60}
{"Display_Name":"token","Creation_Time":"2022-07-25T16:51","Policies":["default","dev"],"Accessor":"56TIiaEnF7woIRba8wZotOmb.Pubjt","ttl":60}
{"Display_Name":"token","Creation_Time":"2022-07-25T16:51","Policies":["default","dev"],"Acces

CLI option

```shell
printf "\n#==> Lookup accessors in variable and save data to file.\n"
printf "#==> Exclude tokens with root policy.\n\n"
time while read accessor; do
# echo vault token lookup -format=json -accessor $accessor   #debug
vault token lookup -format=json -accessor $accessor \
  | jq -c .data \
  | grep -Ev "admin|root" \
  | jq -c 'select((.display_name == "root"|not) and .ttl > 60 ) ' \
  | jq -c '{Display_Name: .display_name, 
  Creation_Time:(.creation_time | strflocaltime("%Y-%m-%dT%H:%M")),
  Policies: .policies, Accessor: .accessor, ttl: (.ttl/60 | round ) }'
done < /tmp/vault_accessors.out | tee /tmp/vault_accessors_delete.out > /dev/null
```

Check to make sure `vault_accessors_delete.out` file got populated.

In [197]:
printf "\n#==> Show a few lines from the file\n"
head /tmp/vault_accessors_delete.out
printf "\n#==> Make sure nothing with root/admin slipped through\n"
grep -e "root | admin" /tmp/vault_accessors_delete.out || echo "nada"


#==> Show a few lines from the file
{"Display_Name":"token","Creation_Time":"2022-07-25T16:52","Policies":["default","dev"],"Accessor":"FV44XiYgcERyYLFWnA7rNzdY.Pubjt","ttl":61}
{"Display_Name":"token","Creation_Time":"2022-07-25T16:52","Policies":["default","dev"],"Accessor":"zZzI5ldkXptUVN0UoXgz9NJ7.Pubjt","ttl":61}
{"Display_Name":"token","Creation_Time":"2022-07-25T16:51","Policies":["default","ops"],"Accessor":"dYsoNQUZO9kxmG4ySTlfNZTP.Pubjt","ttl":60}
{"Display_Name":"token","Creation_Time":"2022-07-25T16:51","Policies":["default","dev"],"Accessor":"F979MwQBYxVmUHhU2BocgKXx.Pubjt","ttl":60}
{"Display_Name":"token","Creation_Time":"2022-07-25T16:51","Policies":["default","dev"],"Accessor":"56TIiaEnF7woIRba8wZotOmb.Pubjt","ttl":60}
{"Display_Name":"token","Creation_Time":"2022-07-25T16:51","Policies":["default","dev"],"Accessor":"Z4xk9t8rlWfOkYvNe7ocKUnh.Pubjt","ttl":60}
{"Display_Name":"token","Creation_Time":"2022-07-25T16:51","Policies":["default","dev"],"Accessor":"VFaIfAUra0l

### Revoke tokens based on filtered list

API option

In [198]:
printf "\n#==> Lookup accessors in variable and save data to file.\n"
printf "#==> Exclude tokens with root policy.\n\n"
time while read accessor; do
curl -s \
  --header "X-Vault-Token: $VAULT_TOKEN" \
  --header "X-Vault-Namespace: $VAULT_NAMESPACE" \
  --request POST \
  --data "{\"accessor\": \"$accessor\"}" \
  ${VAULT_ADDR}/v1/auth/token/revoke-accessor
done < <(jq -r .Accessor /tmp/vault_accessors_delete.out) | tee /tmp/vault_revoke.log
tail  /tmp/vault_revoke.log


#==> Lookup accessors in variable and save data to file.
#==> Exclude tokens with root policy.


real	0m7.666s
user	0m3.032s
sys	0m2.895s


> For API - you will get error if accessor is not valid.
> `{"errors":["1 error occurred:\n\t* invalid accessor\n\n"]}`

> For CLI - you will get an opaque message.
> `Success! Revoked token (if it existed)`


CLI option

```shell
printf "#==> Revoke tokens. Be careful of any tokens with special privileges.\n"
jq -r .Accessor /tmp/vault_accessors_delete.out | while read delete; do
  vault token revoke -accessor $delete > /tmp/vault_revoke.log
done

printf "#==> Show log of revoke activity.\n"
tail /tmp/vault_revoke.log
```

Sample Output
```
#==> Revoke tokens. Be careful of any tokens with special privileges.
Success! Revoked token (if it existed)
Success! Revoked token (if it existed)
Success! Revoked token (if it existed)
Success! Revoked token (if it existed)
Success! Revoked token (if it existed)
Success! Revoked token (if it existed)
Success! Revoked token (if it existed)
Success! Revoked token (if it existed)
Success! Revoked token (if it existed)
Success! Revoked token (if it existed)
Success! Revoked token (if it existed)
```

### how many left

See how many items are left. Check the next 50 items.

API Option

In [199]:
printf "\n#==> Get list of token accessors and place into variable\n"
time curl -s \
  --header "X-Vault-Token: $VAULT_TOKEN" \
  --header "X-Vault-Namespace: $VAULT_NAMESPACE" \
  --request LIST \
  ${VAULT_ADDR}/v1/auth/token/accessors | jq -r .data.keys[] \
  > /tmp/vault_accessors_remaining.out

printf "\n#==> Number of accessors remaining:
$(cat /tmp/vault_accessors_remaining.out | wc -l)\n\n"


#==> Get list of token accessors and place into variable

real	0m0.036s
user	0m0.025s
sys	0m0.010s

#==> Number of accessors remaining:
2450



CLI option

```shell
printf "\n#==> Show all token accessors in current namespace\n"
vault list auth/token/accessors | grep -v admin | wc -l
vault list auth/token/accessors | head -n 52 | tail -n +3 | while read accessor; do
vault token lookup -format=json -accessor $accessor \
  | jq -c ".data | {Display_Name: .display_name, Creation_Time:.creation_time, 
  Policies: .policies, Accessor: .accessor,
  ttl: (.ttl/60 | round )}"
done
```

Sample
```shell
#==> Show all token accessors in current namespace
26
{"Display_Name":"token","Creation_Time":1658190366,"Policies":["root"],"Accessor":"2mvhslx6DCngBa4URtJnTdVa","ttl":0}
{"Display_Name":"token","Creation_Time":1658260574,"Policies":["admin","default","ops"],"Accessor":"3j4PCfh6LrGBUnqTsnwq7pMH","ttl":51}
{"Display_Name":"token","Creation_Time":1658260574,"Policies":["admin","default","ops"],"Accessor":"5mYcgY1vjcgXLTX9L0qsSO28","ttl":51}
{"Display_Name":"token","Creation_Time":1658260573,"Policies":["admin","default"],"Accessor":"E80ZcAgPrinBEjkdfgfUnz9C","ttl":51}
{"Display_Name":"token","Creation_Time":1658260573,"Policies":["admin","default","ops"],"Accessor":"KguRtymstS5oPgHcRe1ODesf","ttl":51}
{"Display_Name":"token","Creation_Time":1658260574,"Policies":["admin","default","ops"],"Accessor":"MfAQeLb8IYBVoBqkrNQLDhQ5","ttl":51}
<snip>
```

### tidy

API option

In [200]:
curl \
    --header "X-Vault-Token: $VAULT_TOKEN" \
    --header "X-Vault-Namespace: $VAULT_NAMESPACE" \
    --request POST \
    $VAULT_ADDR/v1/auth/token/tidy | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   272  100   272    0     0   265k      0 --:--:-- --:--:-- --:--:--  265k
{
  "request_id": "eb9ff7df-436e-2bdc-2fa2-9b0be3246573",
  "lease_id": "",
  "renewable": false,
  "lease_duration": 0,
  "data": null,
  "wrap_info": null,
  "warnings": [
    "Tidy operation successfully started. Any information from the operation will be printed to Vault's server logs."
  ],
  "auth": null
}


vault token tidy -h

Check logs. Sample here is for docker logs.

In [201]:
docker logs vault 2>&1 | grep token | tail -n 20 | awk -F'\\[INFO\\]  ' '{print $NF}'

expiration: revoked lease: lease_id=auth/token/create/h46a1d1a8cc3ce440d294e216ddb075e26a3d988151dc48e7b2335b501bfe0d67.Pubjt
expiration: revoked lease: lease_id=auth/token/create/h778782ca2e3c387b21cb124679df8200f12e5d9d2f76eab8f491f77c8a67add8.Pubjt
expiration: revoked lease: lease_id=auth/token/create/hbd3fe26cda1e717e15f6fa7d351082f506e8eb2a3b60f4b0b3394366a96d7210.Pubjt
expiration: revoked lease: lease_id=auth/token/create/h53c3d973e6a3616cc77775c223e60bbc1fbdf76068eacf2cb70abc0e6ac9c47b.Pubjt
expiration: revoked lease: lease_id=auth/token/create/h72b2b0a8a979d5b872a2f66b474284ac0d2caaa779766217cbfec0a34172745b.Pubjt
token: beginning tidy operation on tokens
token: checking if accessors contain valid tokens: progress=500 percent_complete=20.367346938775512
token: checking if accessors contain valid tokens: progress=1000 percent_complete=40.775510204081634
token: checking if accessors contain valid tokens: progress=1500 percent_complete=61.183673469387756
token: checking if accesso

Sample Output
```shell
[INFO]  token: beginning tidy operation on tokens
[INFO]  token: number of entries scanned in parent prefix: count=0
[INFO]  token: number of entries deleted in parent prefix: count=0
[INFO]  token: number of tokens scanned in parent index list: count=0
[INFO]  token: number of tokens revoked in parent index list: count=0
[INFO]  token: number of accessors scanned: count=0
[INFO]  token: number of deleted accessors which had empty tokens: count=0
[INFO]  token: number of revoked tokens which were invalid but present in accessors: count=0
[INFO]  token: number of deleted accessors which had invalid tokens: count=0
[INFO]  token: number of deleted cubbyhole keys that were invalid: count=0
[INFO]  token: finished tidy operation on tokens
```

Rinse and repeat to delete more.